In [13]:
import sys
from nplinker import NPLinker
from logconfig import LogConfig
from metabolomics import Spectrum

In [14]:
# configuring NPLinker in a notebook env is now done either by passing in the name of a config file,
# or by passing in a dict which corresponds to the structure of the config file. Usually it will be
# easier to edit the file and simply pass the filename like this:
npl = NPLinker('latest_api_demo.toml')

# the above step will attempt to discover the files to be loaded from the dataset and complain
# if they're not as expected. Next, actually load the data files
if not npl.load_data():
    raise Exception('Failed to load data')
 

Loaded 5930 molecules
Found 1816 MiBIG json files
cluster files: /mnt/archive/nplinker_data/CrusemannDataForNPLinker/bigscape/NRPS/NRPS_clustering*
cluster files: /mnt/archive/nplinker_data/CrusemannDataForNPLinker/bigscape/Others/Others_clustering*
cluster files: /mnt/archive/nplinker_data/CrusemannDataForNPLinker/bigscape/PKSI/PKSI_clustering*
cluster files: /mnt/archive/nplinker_data/CrusemannDataForNPLinker/bigscape/PKS-NRP_Hybrids/PKS-NRP_Hybrids_clustering*
2019-03-05 15:59:28,080, WARNING, nplinker.py:174, Missing tsv file for folder: PKS-NRP_Hybrids
cluster files: /mnt/archive/nplinker_data/CrusemannDataForNPLinker/bigscape/PKSother/PKSother_clustering*
cluster files: /mnt/archive/nplinker_data/CrusemannDataForNPLinker/bigscape/RiPPs/RiPPs_clustering*
cluster files: /mnt/archive/nplinker_data/CrusemannDataForNPLinker/bigscape/Saccharides/Saccharides_clustering*
cluster files: /mnt/archive/nplinker_data/CrusemannDataForNPLinker/bigscape/Terpene/Terpene_clustering*
filename /mnt/

In [7]:
# The scoring methods are defined and configured in the default configuration file at 
# ~/.config/nplinker/nplinker.toml, but will be overridden by the config file you loaded above,
# and the scoring methods can be easily changed once the NPLinker object has been created, e.g.:

# ensure only metcalf scoring is enabled, and set a 99% significance percentile threshold
print('Currently enabled scoring methods: {}'.format(npl.scoring.enabled()))
npl.scoring.likescore.enabled = False
# npl.scoring.likescore.cutoff = <scoring cutoff threshold>
npl.scoring.hg.enabled = False
# npl.scoring.hg.prob = <probability threshold>
npl.scoring.metcalf.enabled = True
npl.scoring.metcalf.sig_percentile = 99
print('Currently enabled scoring methods: {}'.format(npl.scoring.enabled()))

Currently enabled scoring methods: [namespace(enabled=True, name='metcalf', sig_percentile=99), namespace(cutoff=100, enabled=True, name='likescore'), namespace(enabled=True, name='hg', prob=0.99)]
Currently enabled scoring methods: [namespace(enabled=True, name='metcalf', sig_percentile=99)]


In [11]:
# this step generates scores for all objects and enabled scoring methods, so it can be
# quite lengthy. The random_count parameter determines the number of randomised instances
# of Spectrum <=> Strain mappings that will be generated during the process.
if not npl.process_dataset(random_count=10):
    raise Exception('Failed to process dataset')
print('Completed generating scores')

Completed generating scores


In [12]:
# to get results once the scores are generated, first select an object you're interested 
# in, then call get_links with a specific scoring method. You can also pass a list of 
# objects as the first parameter. The method returns a list which contains only those
# objects that satisfy the scoring criteria (so here only those with a significance 
# percentile score of >= 99 as set above)
test_gcf = npl.gcfs[8]
results = npl.get_links(test_gcf, npl.scoring.metcalf)
if test_gcf not in results:
    print('No results found!')
else:
    print('Found results for {}!'.format(test_gcf))
    # to get the objects that scored highly against this GCF, use links_for_obj. By
    # default it will return all objects, the type_ parameter can be used to filter
    # by class, so here it will only return spectra
    test_gcf_links = npl.links_for_obj(test_gcf, npl.scoring.metcalf, type_=Spectrum)
    
    # print the objects and their scores, plus common strains
    for obj, score in test_gcf_links:
        print('{} : score {}'.format(obj, score))
        # returns a dict indexed by (Spectrum, GCF) tuples, with 
        # the values being lists of strain names shared between the two
        common_strains = npl.get_common_strains(test_gcf, obj)
        strain_names = list(common_strains.values())[0]
        if len(strain_names) > 0:
            print('   {} shared strains: {}'.format(len(strain_names), strain_names))
        else:
            print('   (no shared strains)')
            
    print('{} total links found'.format(len(test_gcf_links)))
        
    

2019-03-05 15:47:50,563, WARNING, nplinker.py:394, FIXME: MolecularFamily support
Found results for GCF(id=8, short_gcf_id=NRPS_clustering_c0.30.tsv:179)!
Spectrum(id=2620, spectrum_id=56637) : score 160.0
   2 shared strains: ['CNS991', 'CNY231']
Spectrum(id=3034, spectrum_id=63435) : score 160.0
   2 shared strains: ['CNS991', 'CNY231']
Spectrum(id=4420, spectrum_id=84773) : score 160.0
   2 shared strains: ['CNS991', 'CNY486']
Spectrum(id=4923, spectrum_id=95828) : score 160.0
   2 shared strains: ['CNS991', 'CNY231']
Spectrum(id=5288, spectrum_id=100017) : score 160.0
   2 shared strains: ['CNS991', 'CNY231']
Spectrum(id=47, spectrum_id=594) : score 150.0
   1 shared strains: ['CNY231']
Spectrum(id=48, spectrum_id=599) : score 150.0
   1 shared strains: ['CNY231']
Spectrum(id=50, spectrum_id=606) : score 150.0
   1 shared strains: ['CNY231']
Spectrum(id=55, spectrum_id=628) : score 150.0
   1 shared strains: ['CNY231']
Spectrum(id=56, spectrum_id=634) : score 150.0
   1 shared stra